In [10]:
# !pip install ollama gradio pandas

In [11]:
import random
import gradio as gr
import ollama
from dataclasses import dataclass

MSG_REQUIRE_CODE = "❗ 먼저 비밀 코드를 입력하고 시작해주세요."
MSG_INVALID_CODE = "❗ 비밀 코드는 중복 없는 5자리 숫자이며 첫 숫자는 0이 될 수 없습니다."
MSG_INVALID_GUESS = "❗ 5자리 숫자를 정확히 입력하세요. 예: 12345"

@dataclass
class GameState:
    ai_code: str
    logs: list
    user_attempts: int
    ai_attempts: int
    user_won: bool
    ai_won: bool

def initialize():
    first_digit = random.choice("123456789")
    ai_code = first_digit + "".join(random.sample([d for d in "0123456789" if d != first_digit], 4))
    return GameState(
        ai_code=ai_code,
        logs=[],
        user_attempts=0,
        ai_attempts=0,
        user_won=False,
        ai_won=False,
    )

def format_feedback(exact, partial):
    if exact == 0 and partial == 0:
        return "아웃"
    return f"{exact} 스트라이크, {partial} 볼"

def validate_code_input(code):
    return code.isdigit() and len(code) == 5 and code[0] != '0' and len(set(code)) == 5

def validate_guess_input(guess):
    return guess.isdigit() and len(guess) == 5

def compare_codes(guess, target):
    exact = sum(guess[i] == target[i] for i in range(5))
    partial = sum(min(guess.count(d), target.count(d)) for d in set(guess)) - exact
    return exact, partial

def get_ai_guess(history_text):
    prompt = f"""
[게임 룰]
- 당신과 사용자는 각각 중복 없는 5자리 숫자 코드를 설정합니다 (첫 숫자는 0이 될 수 없음)
- 양쪽은 번갈아가며 서로의 코드를 추리합니다
- 피드백은 '스트라이크(숫자+위치 일치)', '볼(숫자만 일치)', '아웃(불일치)'로 주어집니다

당신은 5자리 숫자 코드브레이커 AI입니다. 숫자는 0~9로 이루어지며 중복되지 않습니다.
첫 숫자는 0이 될 수 없습니다.
지금까지의 기록:
{history_text}
예시 형식 (이 숫자는 사용하지 마세요): 38429
"""
    for _ in range(5):
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=[{"role": "user", "content": prompt}])
        guess = ''.join(filter(str.isdigit, response["message"]["content"].strip()))
        if validate_code_input(guess):
            return guess
    return "12345"

def process_turn(user_guess, user_code, state: GameState):
    if user_code.strip() == "":
        return MSG_REQUIRE_CODE, user_code, state.ai_code, state.user_attempts, state.ai_attempts, state.user_won, state.ai_won, state.logs, gr.update(interactive=False), state.logs
    if not validate_code_input(user_code):
        return MSG_INVALID_CODE, user_code, state.ai_code, state.user_attempts, state.ai_attempts, state.user_won, state.ai_won, state.logs, gr.update(interactive=False), state.logs
    if not validate_guess_input(user_guess):
        return MSG_INVALID_GUESS, user_code, state.ai_code, state.user_attempts, state.ai_attempts, state.user_won, state.ai_won, state.logs, gr.update(interactive=False), state.logs

    state.user_attempts += 1
    user_turn_number = state.user_attempts
    exact_u, partial_u = compare_codes(user_guess, state.ai_code)
    feedback_u = format_feedback(exact_u, partial_u)
    state.logs.append({"턴": user_turn_number, "추측자": "👤 사용자", "추측": user_guess, "결과": feedback_u})
    if user_guess == state.ai_code:
        state.user_won = True
        state.logs.append({"턴": "✅", "추측자": "👤 사용자", "추측": user_guess, "결과": f"정답! ({state.user_attempts}회 시도)"})

    log_text = "\n".join([f"{log['추측자']}: {log['추측']} → {log['결과']}" for log in state.logs])
    ai_guess = get_ai_guess(log_text)
    state.ai_attempts += 1
    ai_turn_number = state.ai_attempts
    exact_a, partial_a = compare_codes(ai_guess, user_code)
    feedback_a = format_feedback(exact_a, partial_a)
    state.logs.append({"턴": ai_turn_number, "추측자": "🤖 AI", "추측": ai_guess, "결과": feedback_a})
    if ai_guess == user_code:
        state.ai_won = True
        state.logs.append({"턴": "✅", "추측자": "🤖 AI", "추측": ai_guess, "결과": f"정답! ({state.ai_attempts}회 시도)"})

    import pandas as pd
    return "", user_code, state.ai_code, state.user_attempts, state.ai_attempts, state.user_won, state.ai_won, state.logs, gr.update(interactive=False), pd.DataFrame(state.logs)

def wrapped_process_turn(user_guess, user_code, *state_vars):
    ai_code, user_attempts, ai_attempts, user_won, ai_won, logs = state_vars
    state = GameState(ai_code=ai_code, logs=logs, user_attempts=user_attempts, ai_attempts=ai_attempts, user_won=user_won, ai_won=ai_won)
    result, user_code, ai_code, user_attempts, ai_attempts, user_won, ai_won, logs, user_code_display, output_logs = process_turn(user_guess, user_code, state)
    return result, user_code, ai_code, user_attempts, ai_attempts, user_won, ai_won, logs, user_code_display, output_logs

def reset_game():
    state = initialize()
    return "", state.ai_code, state.user_attempts, state.ai_attempts, state.user_won, state.ai_won, state.logs, gr.update(interactive=True), []

def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 코드브레이커: 사용자 vs AI")

        user_code = gr.Textbox(label="🔐 당신의 비밀 코드", interactive=True, lines=1)
        user_guess = gr.Textbox(label="🎯 이번 턴의 추측", lines=1, interactive=True, autofocus=True, placeholder="예: 12345")
        output_table = gr.Dataframe(headers=["턴", "추측자", "추측", "결과"], datatype=["str", "str", "str", "str"], interactive=False)
        next_btn = gr.Button("🔄 턴 진행")
        reset_btn = gr.Button("🔁 게임 초기화")

        initial_state = initialize()
        state = [gr.State(initial_state.ai_code), gr.State(initial_state.user_attempts), gr.State(initial_state.ai_attempts), gr.State(initial_state.user_won), gr.State(initial_state.ai_won), gr.State(initial_state.logs)]

        next_btn.click(fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[user_guess, user_code, *state, user_code, output_table]
        )

        user_guess.submit(
            fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[user_guess, user_code, *state, user_code, output_table]
        )

        reset_btn.click(
            fn=reset_game,
            inputs=[],
            outputs=[user_guess, *state, user_code, output_table]
        )

    return demo

ui().launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://0636f02dc109b2420c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
